# Title
[]()

In [1]:

import sys
sys.path.append('../src')
sys.path.append(r"/home/silvhua/custom_python")
from silvhua import *

# Initialize

In [2]:
api_key = os.getenv('api_ncbi') # Pubmed API key
result_dict = dict()

# Iteration 1

In [3]:
import sys
sys.path.append(r"/home/silvhua/custom_python")
import os
import pandas as pd
import string
import re
import requests
# from article_processing import create_text_dict_from_folder
# from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

import sys
import os
import requests
from Custom_Logger import *

class Pubmed_API:
    def __init__(self, api_key=os.getenv('api_ncbi'), logger=None, logging_level=logging.INFO):
        self.api_key = api_key
        self.logger = create_function_logger('Pubmed_API', logger, level=logging_level)
        self.iteration = 1
        self.responses_dict = {}
        self.results_dict = {}
        self.PMIDs_dict = {}
        self.record_strings_dict = {}

    def search_article(self, query, query_tag=None, publication=None, reldate=None, retmax=None,
        systematic_only=False, review_only=False, verbose=False, additional_search_params=None
        ):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        response = {}
        results = pd.DataFrame()
        search_term = f'"{re.sub(r"not", "", query)}"'  # Remove 'not' since it will be treated as a boolean
        if query_tag:
            search_term += f'{query_tag}'
        if publication:
            search_term = f'AND {publication} [ta]'
        if systematic_only:
            search_term += ' AND systematic[sb]'
        elif review_only:
            search_term += ' AND (systematic[sb] OR review[pt])'
        params = {
            'db': 'pubmed',
            'term': search_term,
            'retmax': 5,
            'retmode': 'json',
            'datetype': 'edat',
        }
        if reldate:
            params['reldate'] = reldate
        if retmax:
            params['retmax'] = retmax
        if additional_search_params:
            params.update(additional_search_params)
        self.logger.info(f'Search term: {search_term}')
        try:
            response = requests.get(base_url, params=params)
            response_dict = response.json()
            self.responses_dict[self.iteration] = response_dict
            result_dict = self.get_article_data_by_title()
            self.results_dict[self.iteration] = result_dict
            self.iteration += 1
            results = pd.DataFrame(result_dict).transpose()
        except Exception as error:
            error_messages = []
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        
        return results

    def get_article_data_by_title(self):
        try:
            result_dict = {}
            record_strings_list = self.batch_retrieve_citation(self.responses_dict[self.iteration])
            self.record_strings_dict[self.iteration] = record_strings_list
            for index, record_string in enumerate(record_strings_list):
                result_dict[index] = self.extract_pubmed_details(record_string)

        except Exception as error:
            error_messages = []
            error_messages.append(f'Response: \n{self.PMIDs_dict[self.iteration]}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        return result_dict

    def batch_retrieve_citation(self, response_dict):
        result_list = []
        messages = []
        try:
            id_list = response_dict['esearchresult']['idlist']
            self.PMIDs_dict[self.iteration] = id_list
            if id_list:
                self.logger.info(f'Extracting these {len(id_list)} PMIDs: {id_list}')
                for index, id in enumerate(id_list):
                    result_list.append(self.retrieve_citation(id).decode('utf-8'))
                    current_index, current_id = index+1, id
            else:
                self.logger.warning(f'No results found.')
        except Exception as error:
            messages.append(f'Response: \n{response_dict}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            messages.append(f'\tAn error occurred on line {lineno} in {filename}: {error}')
            messages.append(f'Article {current_index} [{current_id}] not found.')
        return result_list

    def retrieve_citation(self, article_id):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        params = {
            'db': 'pubmed',
            'id': article_id
        }

        response = requests.get(base_url, params=params)
        return response.content

    def extract_pubmed_details(self, record_string):
        """
        Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
        """
        authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
        formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

        # Extract publication year
        publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
        publication_year = publication_year.group(1) if publication_year else ''
        publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
        publication_month = publication_month.group(1) if publication_month else ''

        # Extract article title
        article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
        article_title = article_title.group(1) if article_title else ''

        # Extract journal title
        journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
        journal_title = journal_title.group(1) if journal_title else ''

        # Extract journal volume
        journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
        journal_volume = journal_volume.group(1) if journal_volume else ''

        # Extract journal issue
        journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
        journal_issue = journal_issue.group(1) if journal_issue else ''

        # Extract start page
        start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
        start_page = start_page.group(1) if start_page else ''

        # Extract end page
        end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
        end_page = end_page.group(1) if end_page else ''

        # Extract ELocationID
        doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
        doi = doi.group(1) if doi else ''

        # Extract PMID
        pmid = re.search(r'<PMID.*?>(.*?)</PMID>', record_string)
        pmid = pmid.group(1) if pmid else ''

        abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
        self.logger.debug(f'Number of abstract sections: {len(abstract_matches)}')
        if len(abstract_matches) > 1:
            cleaned_abstract_sections = []
            for match in abstract_matches:
                clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
                clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
                cleaned_abstract_sections.append(clean_match)
                
            abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
        else:
            abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''
            
        # Extract MeshHeadingList
        MeshHeadingList = re.search(r'<MeshHeadingList>(.*?)</MeshHeadingList>', record_string)
        MeshHeadingList = MeshHeadingList.group(1) if MeshHeadingList else ''
        
        return {
            'pubmed_title': article_title,
            'abstract': abstract,
            'journal': journal_title,
            'authors': formatted_authors,
            'year': publication_year,
            'month': publication_month,
            'pub_volume': journal_volume,
            'pub_issue': journal_issue,
            'start_page': start_page,
            'end_page': end_page,
            'doi': doi,
            'pmid': pmid,
            'mesh_headings': MeshHeadingList
        }
    
iteration = 1
query = '("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
result_dict[iteration].search_article(query, retmax=2)

2024-04-03 14:30:09,078 - Pubmed_API - INFO:
Search term: "("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))"

2024-04-03 14:30:09,942 - Pubmed_API - WARNING:
No results found.



""


In [5]:
iteration = 1.1
query = '("resistance train*"[All Fields]) AND (AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
result_dict[iteration].search_article(query, retmax=2)

2024-04-03 14:34:34,405 - Pubmed_API - INFO:
Search term: "("resistance train*"[All Fields]) AND (AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))"

2024-04-03 14:34:35,532 - Pubmed_API - WARNING:
No results found.



""


In [6]:
iteration = 1.2
query = '("resistance train*")'
result_dict[iteration] = Pubmed_API()
result_dict[iteration].search_article(query, retmax=2)

2024-04-03 14:35:07,969 - Pubmed_API - INFO:
Search term: "("resistance train*")"

2024-04-03 14:35:08,540 - Pubmed_API - WARNING:
No results found.



""


In [7]:
iteration = 1.2
query = 'resistance train*'
result_dict[iteration] = Pubmed_API()
result_dict[iteration].search_article(query, retmax=2)

2024-04-03 14:35:35,164 - Pubmed_API - INFO:
Search term: "resistance train*"

2024-04-03 14:35:35,897 - Pubmed_API - INFO:
Extracting these 2 PMIDs: ['38568258', '38565633']



,pubmed_title,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,pmid,mesh_headings
0,Is there evidence for the asymmetrical transfe...,PURPOSE: The literature predominantly addresse...,European journal of applied physiology,"Vickie Wong, Jun Seob Song, Yujiro Yamada, Ryo...",2024,,,,,,10.1007/s00421-024-05472-9,38568258,
1,Effect of resistance training plus enriched pr...,This study aimed to determine the effects of r...,Scientific reports,"Majid Mohabbat, Hamid Arazi",2024,,14,1,7744,,10.1038/s41598-024-58462-4,38565633,


In [8]:
iteration = 1.3
query = '"resistance train*"'
result_dict[iteration] = Pubmed_API()
result_dict[iteration].search_article(query, retmax=2)

2024-04-03 14:35:59,787 - Pubmed_API - INFO:
Search term: ""resistance train*""

2024-04-03 14:36:00,439 - Pubmed_API - INFO:
Extracting these 2 PMIDs: ['38568258', '38567973']



,pubmed_title,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,pmid,mesh_headings
0,Is there evidence for the asymmetrical transfe...,PURPOSE: The literature predominantly addresse...,European journal of applied physiology,"Vickie Wong, Jun Seob Song, Yujiro Yamada, Ryo...",2024,,,,,,10.1007/s00421-024-05472-9,38568258,
1,Are train horns improving road safety? Road us...,: Train horns are used as a control at railway...,Ergonomics,"Gr&#xe9;goire S Larue, Danielle Villoresi, Son...",2024,,,,1,12,10.1080/00140139.2024.2333965,38567973,


# iteration

In [10]:
import sys
sys.path.append(r"/home/silvhua/custom_python")
import os
import pandas as pd
import string
import re
import requests
# from article_processing import create_text_dict_from_folder
# from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

import sys
import os
import requests
from Custom_Logger import *

class Pubmed_API:
    def __init__(self, api_key=os.getenv('api_ncbi'), logger=None, logging_level=logging.INFO):
        self.api_key = api_key
        self.logger = create_function_logger('Pubmed_API', logger, level=logging_level)
        self.iteration = 1
        self.responses_dict = {}
        self.results_dict = {}
        self.PMIDs_dict = {}
        self.record_strings_dict = {}

    def search_article(self, query, query_tag=None, publication=None, reldate=None, retmax=None,
        systematic_only=False, review_only=False, additional_search_params=None, ids_only=False
        ):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        response = {}
        results = pd.DataFrame()
        search_term = f'{re.sub(r"not", "", query)}'  # Remove 'not' since it will be treated as a boolean
        if query_tag:
            search_term += f'{query_tag}'
        if publication:
            search_term = f'AND {publication} [ta]'
        if systematic_only:
            search_term += ' AND systematic[sb]'
        elif review_only:
            search_term += ' AND (systematic[sb] OR review[pt])'
        params = {
            'db': 'pubmed',
            'term': search_term,
            'retmax': 5,
            'retmode': 'json',
            'datetype': 'edat',
        }
        if reldate:
            params['reldate'] = reldate
        if retmax:
            params['retmax'] = retmax
        if additional_search_params:
            params.update(additional_search_params)
        self.logger.info(f'Search term: {search_term}')
        try:
            response = requests.get(base_url, params=params)
            response_dict = response.json()
            self.responses_dict[self.iteration] = response_dict
            if ids_only==False:
                result_dict = self.get_article_data_by_title()
                self.results_dict[self.iteration] = result_dict
                self.iteration += 1
                results = pd.DataFrame(result_dict).transpose()
            else:
                results = response_dict
        except Exception as error:
            error_messages = []
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        
        return results

    def get_article_data_by_title(self):
        try:
            result_dict = {}
            record_strings_list = self.batch_retrieve_citation(self.responses_dict[self.iteration])
            self.record_strings_dict[self.iteration] = record_strings_list
            for index, record_string in enumerate(record_strings_list):
                result_dict[index] = self.extract_pubmed_details(record_string)

        except Exception as error:
            error_messages = []
            error_messages.append(f'Response: \n{self.PMIDs_dict[self.iteration]}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        return result_dict

    def batch_retrieve_citation(self, response_dict):
        result_list = []
        messages = []
        try:
            id_list = response_dict['esearchresult']['idlist']
            self.PMIDs_dict[self.iteration] = id_list
            if id_list:
                self.logger.info(f'Extracting these {len(id_list)} PMIDs: {id_list}')
                for index, id in enumerate(id_list):
                    result_list.append(self.retrieve_citation(id).decode('utf-8'))
                    current_index, current_id = index+1, id
            else:
                self.logger.warning(f'No results found.')
        except Exception as error:
            messages.append(f'Response: \n{response_dict}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            messages.append(f'\tAn error occurred on line {lineno} in {filename}: {error}')
            messages.append(f'Article {current_index} [{current_id}] not found.')
        return result_list

    def retrieve_citation(self, article_id):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        params = {
            'db': 'pubmed',
            'id': article_id
        }
        response = requests.get(base_url, params=params)
        return response.content

    def extract_pubmed_details(self, record_string):
        """
        Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
        """
        authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
        formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

        # Extract publication year
        publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
        publication_year = publication_year.group(1) if publication_year else ''
        publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
        publication_month = publication_month.group(1) if publication_month else ''

        # Extract article title
        article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
        article_title = article_title.group(1) if article_title else ''

        # Extract journal title
        journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
        journal_title = journal_title.group(1) if journal_title else ''

        # Extract journal volume
        journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
        journal_volume = journal_volume.group(1) if journal_volume else ''

        # Extract journal issue
        journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
        journal_issue = journal_issue.group(1) if journal_issue else ''

        # Extract start page
        start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
        start_page = start_page.group(1) if start_page else ''

        # Extract end page
        end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
        end_page = end_page.group(1) if end_page else ''

        # Extract ELocationID
        doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
        doi = doi.group(1) if doi else ''

        # Extract PMID
        pmid = re.search(r'<PMID.*?>(.*?)</PMID>', record_string)
        pmid = pmid.group(1) if pmid else ''

        abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
        self.logger.debug(f'Number of abstract sections: {len(abstract_matches)}')
        if len(abstract_matches) > 1:
            cleaned_abstract_sections = []
            for match in abstract_matches:
                clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
                clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
                cleaned_abstract_sections.append(clean_match)
                
            abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
        else:
            abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''
            
        # Extract MeshHeadingList
        MeshHeadingList = re.search(r'<MeshHeadingList>(.*?)</MeshHeadingList>', record_string)
        MeshHeadingList = MeshHeadingList.group(1) if MeshHeadingList else ''
        return {
            'pubmed_title': article_title,
            'abstract': abstract,
            'journal': journal_title,
            'authors': formatted_authors,
            'year': publication_year,
            'month': publication_month,
            'pub_volume': journal_volume,
            'pub_issue': journal_issue,
            'start_page': start_page,
            'end_page': end_page,
            'doi': doi,
            'pmid': pmid,
            'mesh_headings': MeshHeadingList
        }
    
iteration = 2.1
query = '("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
ids_list = result_dict[iteration].search_article(query, retmax=2, ids_only=True)
ids_list

2024-04-03 14:46:28,832 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))



{'header': {'type': 'esearch', 'version': '0.3'},
 'esearchresult': {'count': '2537',
  'retmax': '2',
  'retstart': '0',
  'idlist': ['38563729', '38563037'],
  'translationset': [{'from': 'y_10[Filter]', 'to': '"last 10 years"[dp]'},
   {'from': 'meta-analysis[Filter]', 'to': 'meta-analysis [PT]'},
   {'from': 'review[Filter]', 'to': 'review [PT]'}],
  'querytranslation': '"resistance train*"[All Fields] AND ("2014/04/03 00:00":"3000/01/01 05:00"[Date - Publication] AND ("meta analysis"[Publication Type] OR "review"[Publication Type] OR "systematic review"[Filter]))'}}

# Iteration 2.2

In [12]:
import sys
sys.path.append(r"/home/silvhua/custom_python")
import os
import pandas as pd
import string
import re
import requests
# from article_processing import create_text_dict_from_folder
# from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

import sys
import os
import requests
from Custom_Logger import *

class Pubmed_API:
    def __init__(self, api_key=os.getenv('api_ncbi'), logger=None, logging_level=logging.INFO):
        self.api_key = api_key
        self.logger = create_function_logger('Pubmed_API', logger, level=logging_level)
        self.iteration = 1
        self.responses_dict = {}
        self.results_dict = {}
        self.PMIDs_dict = {}
        self.record_strings_dict = {}

    def search_article(self, query, query_tag=None, publication=None, reldate=None, retmax=None,
        systematic_only=False, review_only=False, additional_search_params=None, ids_only=False
        ):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        response = {}
        results = pd.DataFrame()
        search_term = f'{re.sub(r"not", "", query)}'  # Remove 'not' since it will be treated as a boolean
        if query_tag:
            search_term += f'{query_tag}'
        if publication:
            search_term = f'AND {publication} [ta]'
        if systematic_only:
            search_term += ' AND systematic[sb]'
        elif review_only:
            search_term += ' AND (systematic[sb] OR review[pt])'
        params = {
            'db': 'pubmed',
            'term': search_term,
            'retmax': 5,
            'retmode': 'json',
            'datetype': 'edat',
        }
        if reldate:
            params['reldate'] = reldate
        if retmax:
            params['retmax'] = retmax
        if additional_search_params:
            params.update(additional_search_params)
        self.logger.info(f'Search term: {search_term}')
        try:
            response = requests.get(base_url, params=params)
            response_dict = response.json()
            id_list = response_dict['esearchresult']['idlist']
            self.PMIDs_dict[self.iteration] = id_list
            self.responses_dict[self.iteration] = response_dict
            if ids_only==False:
                result_dict = self.get_article_data_by_title()
                self.results_dict[self.iteration] = result_dict
                self.iteration += 1
                results = pd.DataFrame(result_dict).transpose()
            else:
                results = id_list
        except Exception as error:
            error_messages = []
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        
        return results

    def get_article_data_by_title(self):
        try:
            result_dict = {}
            record_strings_list = self.batch_retrieve_citation()
            self.record_strings_dict[self.iteration] = record_strings_list
            for index, record_string in enumerate(record_strings_list):
                result_dict[index] = self.extract_pubmed_details(record_string)

        except Exception as error:
            error_messages = []
            error_messages.append(f'Response: \n{self.PMIDs_dict[self.iteration]}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        return result_dict

    def batch_retrieve_citation(self, response_dict):
        result_list = []
        messages = []
        try:
            id_list = self.PMIDs_dict.get(self.iteration)
            if id_list:
                self.logger.info(f'Extracting these {len(id_list)} PMIDs: {id_list}')
                for index, id in enumerate(id_list):
                    result_list.append(self.retrieve_citation(id).decode('utf-8'))
                    current_index, current_id = index+1, id
            else:
                self.logger.warning(f'No results found.')
        except Exception as error:
            messages.append(f'Response: \n{response_dict}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            messages.append(f'\tAn error occurred on line {lineno} in {filename}: {error}')
            messages.append(f'Article {current_index} [{current_id}] not found.')
        return result_list

    def retrieve_citation(self, article_id):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        params = {
            'db': 'pubmed',
            'id': article_id
        }
        response = requests.get(base_url, params=params)
        return response.content

    def extract_pubmed_details(self, record_string):
        """
        Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
        """
        authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
        formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

        # Extract publication year
        publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
        publication_year = publication_year.group(1) if publication_year else ''
        publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
        publication_month = publication_month.group(1) if publication_month else ''

        # Extract article title
        article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
        article_title = article_title.group(1) if article_title else ''

        # Extract journal title
        journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
        journal_title = journal_title.group(1) if journal_title else ''

        # Extract journal volume
        journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
        journal_volume = journal_volume.group(1) if journal_volume else ''

        # Extract journal issue
        journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
        journal_issue = journal_issue.group(1) if journal_issue else ''

        # Extract start page
        start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
        start_page = start_page.group(1) if start_page else ''

        # Extract end page
        end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
        end_page = end_page.group(1) if end_page else ''

        # Extract ELocationID
        doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
        doi = doi.group(1) if doi else ''

        # Extract PMID
        pmid = re.search(r'<PMID.*?>(.*?)</PMID>', record_string)
        pmid = pmid.group(1) if pmid else ''

        abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
        self.logger.debug(f'Number of abstract sections: {len(abstract_matches)}')
        if len(abstract_matches) > 1:
            cleaned_abstract_sections = []
            for match in abstract_matches:
                clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
                clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
                cleaned_abstract_sections.append(clean_match)
                
            abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
        else:
            abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''
            
        # Extract MeshHeadingList
        MeshHeadingList = re.search(r'<MeshHeadingList>(.*?)</MeshHeadingList>', record_string)
        MeshHeadingList = MeshHeadingList.group(1) if MeshHeadingList else ''
        return {
            'pubmed_title': article_title,
            'abstract': abstract,
            'journal': journal_title,
            'authors': formatted_authors,
            'year': publication_year,
            'month': publication_month,
            'pub_volume': journal_volume,
            'pub_issue': journal_issue,
            'start_page': start_page,
            'end_page': end_page,
            'doi': doi,
            'pmid': pmid,
            'mesh_headings': MeshHeadingList
        }
    
iteration = 2.2
query = '("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
ids_list = result_dict[iteration].search_article(query, retmax=2, ids_only=True)
ids_list

2024-04-03 14:57:07,071 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))



['38563729', '38563037']

In [13]:
df = result_dict[iteration].get_article_data_by_title()

2024-04-03 14:57:55,977 - Pubmed_API - ERROR:
Response: 
['38563729', '38563037']
	An error occurred on line 86 in /tmp/ipykernel_2364/2201311683.py: Pubmed_API.batch_retrieve_citation() missing 1 required positional argument: 'response_dict'



# Iteration 2.3

In [14]:
import sys
sys.path.append(r"/home/silvhua/custom_python")
import os
import pandas as pd
import string
import re
import requests
# from article_processing import create_text_dict_from_folder
# from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

import sys
import os
import requests
from Custom_Logger import *

class Pubmed_API:
    def __init__(self, api_key=os.getenv('api_ncbi'), logger=None, logging_level=logging.INFO):
        self.api_key = api_key
        self.logger = create_function_logger('Pubmed_API', logger, level=logging_level)
        self.iteration = 1
        self.responses_dict = {}
        self.results_dict = {}
        self.PMIDs_dict = {}
        self.record_strings_dict = {}

    def search_article(self, query, query_tag=None, publication=None, reldate=None, retmax=None,
        systematic_only=False, review_only=False, additional_search_params=None, ids_only=False, 
        verbose=True
        ):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        response = {}
        results = pd.DataFrame()
        search_term = f'{re.sub(r"not", "", query)}'  # Remove 'not' since it will be treated as a boolean
        if query_tag:
            search_term += f'{query_tag}'
        if publication:
            search_term = f'AND {publication} [ta]'
        if systematic_only:
            search_term += ' AND systematic[sb]'
        elif review_only:
            search_term += ' AND (systematic[sb] OR review[pt])'
        params = {
            'db': 'pubmed',
            'term': search_term,
            'retmax': 5,
            'retmode': 'json',
            'datetype': 'edat',
        }
        if reldate:
            params['reldate'] = reldate
        if retmax:
            params['retmax'] = retmax
        if additional_search_params:
            params.update(additional_search_params)
        self.logger.info(f'Search term: {search_term}')
        messages = []
        try:
            response = requests.get(base_url, params=params)
            response_dict = response.json()
            id_list = response_dict['esearchresult']['idlist']
            messages.append(f'{len(id_list)} PMIDs found.')
            if verbose==True:
                messages.append(f'{id_list}')
            self.PMIDs_dict[self.iteration] = id_list
            self.responses_dict[self.iteration] = response_dict
            if ids_only==False:
                result_dict = self.get_article_data_by_title()
                self.results_dict[self.iteration] = result_dict
                self.iteration += 1
                results = pd.DataFrame(result_dict).transpose()
            else:
                results = id_list
        except Exception as error:
            error_messages = []
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        
        return results

    def get_article_data_by_title(self):
        try:
            result_dict = {}
            record_strings_list = self.batch_retrieve_citation()
            self.record_strings_dict[self.iteration] = record_strings_list
            for index, record_string in enumerate(record_strings_list):
                result_dict[index] = self.extract_pubmed_details(record_string)

        except Exception as error:
            error_messages = []
            error_messages.append(f'Response: \n{self.PMIDs_dict[self.iteration]}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        return result_dict

    def batch_retrieve_citation(self):
        result_list = []
        messages = []
        try:
            id_list = self.PMIDs_dict.get(self.iteration)
            if id_list:
                self.logger.info(f'Extracting these {len(id_list)} PMIDs: {id_list}')
                for index, id in enumerate(id_list):
                    result_list.append(self.retrieve_citation(id).decode('utf-8'))
                    current_index, current_id = index+1, id
            else:
                self.logger.warning(f'No results found.')
        except Exception as error:
            messages.append(f'Response: \n{self.response_dict.get(self.iteration)}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            messages.append(f'\tAn error occurred on line {lineno} in {filename}: {error}')
            messages.append(f'Article {current_index} [{current_id}] not found.')
        return result_list

    def retrieve_citation(self, article_id):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        params = {
            'db': 'pubmed',
            'id': article_id
        }
        response = requests.get(base_url, params=params)
        return response.content

    def extract_pubmed_details(self, record_string):
        """
        Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
        """
        authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
        formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

        # Extract publication year
        publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
        publication_year = publication_year.group(1) if publication_year else ''
        publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
        publication_month = publication_month.group(1) if publication_month else ''

        # Extract article title
        article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
        article_title = article_title.group(1) if article_title else ''

        # Extract journal title
        journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
        journal_title = journal_title.group(1) if journal_title else ''

        # Extract journal volume
        journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
        journal_volume = journal_volume.group(1) if journal_volume else ''

        # Extract journal issue
        journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
        journal_issue = journal_issue.group(1) if journal_issue else ''

        # Extract start page
        start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
        start_page = start_page.group(1) if start_page else ''

        # Extract end page
        end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
        end_page = end_page.group(1) if end_page else ''

        # Extract ELocationID
        doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
        doi = doi.group(1) if doi else ''

        # Extract PMID
        pmid = re.search(r'<PMID.*?>(.*?)</PMID>', record_string)
        pmid = pmid.group(1) if pmid else ''

        abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
        self.logger.debug(f'Number of abstract sections: {len(abstract_matches)}')
        if len(abstract_matches) > 1:
            cleaned_abstract_sections = []
            for match in abstract_matches:
                clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
                clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
                cleaned_abstract_sections.append(clean_match)
                
            abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
        else:
            abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''
            
        # Extract MeshHeadingList
        MeshHeadingList = re.search(r'<MeshHeadingList>(.*?)</MeshHeadingList>', record_string)
        MeshHeadingList = MeshHeadingList.group(1) if MeshHeadingList else ''
        return {
            'pubmed_title': article_title,
            'abstract': abstract,
            'journal': journal_title,
            'authors': formatted_authors,
            'year': publication_year,
            'month': publication_month,
            'pub_volume': journal_volume,
            'pub_issue': journal_issue,
            'start_page': start_page,
            'end_page': end_page,
            'doi': doi,
            'pmid': pmid,
            'mesh_headings': MeshHeadingList
        }
    
iteration = 2.3
query = '("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
ids_list = result_dict[iteration].search_article(query, retmax=2, ids_only=True)
df = result_dict[iteration].get_article_data_by_title()
df

2024-04-03 15:03:20,091 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))

2024-04-03 15:03:21,031 - Pubmed_API - INFO:
Extracting these 2 PMIDs: ['38563729', '38563037']



{0: {'pubmed_title': 'Feasibility and Usefulness of Repetitions-In-Reserve Scales for Selecting Exercise Intensity: A Scoping Review.',
  'abstract': 'The intensity of resistance training (RT) exercise is an important consideration for determining relevant health and performance-related outcomes. Yet, current objective exercise intensity measures present concerns in terms of viability or cost. In response to these concerns, repetition-in-reserve (RIR) scales may represent an adequate method of measuring and regulating intensity. However, no recent review has focused on how RIR scales have been used for this purpose in prior research. We prepared the present scoping review to analyze the feasibility and usefulness of RIR scales in selecting RT intensity. We conducted a systematic search in PubMed, SPORTDiscus, PsycINFO, and ClinicalTrials.gov databases (last search date April 2023) for experimental and non-experimental studies that utilized an RIR scale to measure proximity to failure i

# Iteration 2.4

In [19]:
import sys
sys.path.append(r"/home/silvhua/custom_python")
import os
import pandas as pd
import string
import re
import requests
# from article_processing import create_text_dict_from_folder
# from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

import sys
import os
import requests
from Custom_Logger import *

class Pubmed_API:
    def __init__(self, api_key=os.getenv('api_ncbi'), logger=None, logging_level=logging.INFO):
        self.api_key = api_key
        self.logger = create_function_logger('Pubmed_API', logger, level=logging_level)
        self.iteration = 0
        self.responses_dict = {}
        self.results_dict = {}
        self.PMIDs_dict = {}
        self.record_strings_dict = {}

    def search_article(self, query, query_tag=None, publication=None, reldate=None, retmax=None,
        systematic_only=False, review_only=False, additional_search_params=None, ids_only=False, 
        verbose=True
        ):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        response = {}
        results = pd.DataFrame()
        search_term = f'{re.sub(r"not", "", query)}'  # Remove 'not' since it will be treated as a boolean
        if query_tag:
            search_term += f'{query_tag}'
        if publication:
            search_term = f'AND {publication} [ta]'
        if systematic_only:
            search_term += ' AND systematic[sb]'
        elif review_only:
            search_term += ' AND (systematic[sb] OR review[pt])'
        params = {
            'db': 'pubmed',
            'term': search_term,
            'retmax': 5,
            'retmode': 'json',
            'datetype': 'edat',
        }
        if reldate:
            params['reldate'] = reldate
        if retmax:
            params['retmax'] = retmax
        if additional_search_params:
            params.update(additional_search_params)
        self.logger.info(f'Search term: {search_term}')
        messages = []
        try:
            self.iteration += 1
            response = requests.get(base_url, params=params)
            response_dict = response.json()
            id_list = response_dict['esearchresult']['idlist']
            messages.append(f'{len(id_list)} PMIDs found.')
            if verbose==True:
                messages.append(f'{id_list}')
            self.PMIDs_dict[self.iteration] = id_list
            self.responses_dict[self.iteration] = response_dict
            if ids_only==False:
                results = self.get_article_data_by_title()
            else:
                results = id_list
            self.logger.info('\n'.join(messages))
        except Exception as error:
            error_messages = []
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        
        return results

    def get_article_data_by_title(self, iteration=None):
        result_df = pd.DataFrame()
        try:
            result_dict = {}
            iteration = self.iteration if iteration == None else iteration
            record_strings_list = self.batch_retrieve_citation(iteration)
            self.record_strings_dict[iteration] = record_strings_list
            for index, record_string in enumerate(record_strings_list):
                result_dict[index] = self.extract_pubmed_details(record_string)
            self.results_dict[iteration] = result_dict
            result_df = pd.DataFrame(result_dict).transpose()
        except Exception as error:
            error_messages = []
            error_messages.append(f'Response: \n{self.PMIDs_dict.get(iteration)}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            message = f'\tAn error occurred on line {lineno} in {filename}: {error}'
            error_messages.append(message)
            self.logger.error('\n'.join(error_messages))
        return result_df

    def batch_retrieve_citation(self, iteration):
        result_list = []
        messages = []
        try:
            id_list = self.PMIDs_dict.get(iteration)
            if id_list:
                self.logger.info(f'Extracting these {len(id_list)} PMIDs: {id_list}')
                for index, id in enumerate(id_list):
                    result_list.append(self.retrieve_citation(id).decode('utf-8'))
                    current_index, current_id = index+1, id
            else:
                self.logger.warning(f'No results found.')
        except Exception as error:
            messages.append(f'Response: \n{self.responses_dict.get(iteration)}')
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            messages.append(f'\tAn error occurred on line {lineno} in {filename}: {error}')
            messages.append(f'Article {current_index} [{current_id}] not found.')
        return result_list

    def retrieve_citation(self, article_id):
        base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        if self.api_key:
            base_url += f'&api_key={self.api_key}'
        params = {
            'db': 'pubmed',
            'id': article_id
        }
        response = requests.get(base_url, params=params)
        return response.content

    def extract_pubmed_details(self, record_string):
        """
        Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
        """
        authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
        formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

        # Extract publication year
        publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
        publication_year = publication_year.group(1) if publication_year else ''
        publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
        publication_month = publication_month.group(1) if publication_month else ''

        # Extract article title
        article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
        article_title = article_title.group(1) if article_title else ''

        # Extract journal title
        journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
        journal_title = journal_title.group(1) if journal_title else ''

        # Extract journal volume
        journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
        journal_volume = journal_volume.group(1) if journal_volume else ''

        # Extract journal issue
        journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
        journal_issue = journal_issue.group(1) if journal_issue else ''

        # Extract start page
        start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
        start_page = start_page.group(1) if start_page else ''

        # Extract end page
        end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
        end_page = end_page.group(1) if end_page else ''

        # Extract ELocationID
        doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
        doi = doi.group(1) if doi else ''

        # Extract PMID
        pmid = re.search(r'<PMID.*?>(.*?)</PMID>', record_string)
        pmid = pmid.group(1) if pmid else ''

        abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
        self.logger.debug(f'Number of abstract sections: {len(abstract_matches)}')
        if len(abstract_matches) > 1:
            cleaned_abstract_sections = []
            for match in abstract_matches:
                clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
                clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
                cleaned_abstract_sections.append(clean_match)
                
            abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
        else:
            abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''
            
        # Extract MeshHeadingList
        MeshHeadingList = re.search(r'<MeshHeadingList>(.*?)</MeshHeadingList>', record_string)
        MeshHeadingList = MeshHeadingList.group(1) if MeshHeadingList else ''
        return {
            'pubmed_title': article_title,
            'abstract': abstract,
            'journal': journal_title,
            'authors': formatted_authors,
            'year': publication_year,
            'month': publication_month,
            'pub_volume': journal_volume,
            'pub_issue': journal_issue,
            'start_page': start_page,
            'end_page': end_page,
            'doi': doi,
            'pmid': pmid,
            'mesh_headings': MeshHeadingList
        }
    
iteration = 2.4
query = '("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
ids_list = result_dict[iteration].search_article(query, retmax=2, ids_only=True)
df = result_dict[iteration].get_article_data_by_title()
df

2024-04-03 15:24:09,986 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))



2024-04-03 15:24:10,634 - Pubmed_API - INFO:
2 PMIDs found.
['38563729', '38563037']

2024-04-03 15:24:10,640 - Pubmed_API - INFO:
Extracting these 2 PMIDs: ['38563729', '38563037']



,pubmed_title,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,pmid,mesh_headings
0,Feasibility and Usefulness of Repetitions-In-R...,The intensity of resistance training (RT) exer...,Perceptual and motor skills,"Vasco Bastos, S&#xe9;rgio Machado, Diogo S Tei...",2024,,,,315125241241785,,10.1177/00315125241241785,38563729,
1,Differences in the Impact of Various Types of ...,"BACKGROUND: Irisin, a myokine that is responsi...",International journal of preventive medicine,"Atefe Torabi, Jalil Reisi, Mehdi Kargarfard, M...",2024,,15,,11,,10.4103/ijpvm.ijpvm_76_23,38563037,


In [22]:
result_dict[iteration].responses_dict

{1: {'header': {'type': 'esearch', 'version': '0.3'},
  'esearchresult': {'count': '2537',
   'retmax': '2',
   'retstart': '0',
   'idlist': ['38563729', '38563037'],
   'translationset': [{'from': 'y_10[Filter]', 'to': '"last 10 years"[dp]'},
    {'from': 'meta-analysis[Filter]', 'to': 'meta-analysis [PT]'},
    {'from': 'review[Filter]', 'to': 'review [PT]'}],
   'querytranslation': '"resistance train*"[All Fields] AND ("2014/04/03 00:00":"3000/01/01 05:00"[Date - Publication] AND ("meta analysis"[Publication Type] OR "review"[Publication Type] OR "systematic review"[Filter]))'}}}

In [24]:
iteration = 2.41
query = '("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
result_dict[iteration + 0.001] = result_dict[iteration].search_article(query, retmax=2, ids_only=False)
result_dict[iteration + 0.001]

2024-04-03 15:26:34,873 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))

2024-04-03 15:26:35,714 - Pubmed_API - INFO:
Extracting these 2 PMIDs: ['38563729', '38563037']

2024-04-03 15:26:37,223 - Pubmed_API - INFO:
2 PMIDs found.
['38563729', '38563037']



,pubmed_title,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,pmid,mesh_headings
0,Feasibility and Usefulness of Repetitions-In-R...,The intensity of resistance training (RT) exer...,Perceptual and motor skills,"Vasco Bastos, S&#xe9;rgio Machado, Diogo S Tei...",2024,,,,315125241241785,,10.1177/00315125241241785,38563729,
1,Differences in the Impact of Various Types of ...,"BACKGROUND: Irisin, a myokine that is responsi...",International journal of preventive medicine,"Atefe Torabi, Jalil Reisi, Mehdi Kargarfard, M...",2024,,15,,11,,10.4103/ijpvm.ijpvm_76_23,38563037,


## 2.42

In [29]:
iteration = 2.42
query = '("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration] = Pubmed_API()
result_dict[iteration].search_article(query, retmax=2, ids_only=False)

2024-04-03 15:29:54,760 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields]) AND ((y_10[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))



2024-04-03 15:29:55,529 - Pubmed_API - INFO:
Extracting these 2 PMIDs: ['38563729', '38563037']

2024-04-03 15:29:56,677 - Pubmed_API - INFO:
2 PMIDs found.
['38563729', '38563037']



,pubmed_title,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,pmid,mesh_headings
0,Feasibility and Usefulness of Repetitions-In-R...,The intensity of resistance training (RT) exer...,Perceptual and motor skills,"Vasco Bastos, S&#xe9;rgio Machado, Diogo S Tei...",2024,,,,315125241241785,,10.1177/00315125241241785,38563729,
1,Differences in the Impact of Various Types of ...,"BACKGROUND: Irisin, a myokine that is responsi...",International journal of preventive medicine,"Atefe Torabi, Jalil Reisi, Mehdi Kargarfard, M...",2024,,15,,11,,10.4103/ijpvm.ijpvm_76_23,38563037,


In [31]:
result_dict[iteration].PMIDs_dict

{1: ['38563729', '38563037']}

In [32]:
query = '("resistance train*"[All Fields])'
result_dict[iteration].search_article(query, retmax=2, ids_only=False)

2024-04-03 15:30:44,040 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields])

2024-04-03 15:30:44,592 - Pubmed_API - INFO:
Extracting these 2 PMIDs: ['38568258', '38565633']

2024-04-03 15:30:45,777 - Pubmed_API - INFO:
2 PMIDs found.
['38568258', '38565633']



,pubmed_title,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,pmid,mesh_headings
0,Is there evidence for the asymmetrical transfe...,PURPOSE: The literature predominantly addresse...,European journal of applied physiology,"Vickie Wong, Jun Seob Song, Yujiro Yamada, Ryo...",2024,,,,,,10.1007/s00421-024-05472-9,38568258,
1,Effect of resistance training plus enriched pr...,This study aimed to determine the effects of r...,Scientific reports,"Majid Mohabbat, Hamid Arazi",2024,,14,1,7744,,10.1038/s41598-024-58462-4,38565633,


In [34]:
query = '("resistance train*"[All Fields]) AND ((y_1[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))'
result_dict[iteration].search_article(query, ids_only=False)

2024-04-03 15:31:41,653 - Pubmed_API - INFO:
Search term: ("resistance train*"[All Fields]) AND ((y_1[Filter]) AND (meta-analysis[Filter] OR review[Filter] OR systematicreview[Filter]))

2024-04-03 15:31:42,400 - Pubmed_API - INFO:
Extracting these 5 PMIDs: ['38563729', '38563037', '38561438', '38559546', '38549168']

2024-04-03 15:31:44,977 - Pubmed_API - INFO:
5 PMIDs found.
['38563729', '38563037', '38561438', '38559546', '38549168']



,pubmed_title,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,pmid,mesh_headings
0,Feasibility and Usefulness of Repetitions-In-R...,The intensity of resistance training (RT) exer...,Perceptual and motor skills,"Vasco Bastos, S&#xe9;rgio Machado, Diogo S Tei...",2024,,,,315125241241785,,10.1177/00315125241241785,38563729,
1,Differences in the Impact of Various Types of ...,"BACKGROUND: Irisin, a myokine that is responsi...",International journal of preventive medicine,"Atefe Torabi, Jalil Reisi, Mehdi Kargarfard, M...",2024,,15,,11,,10.4103/ijpvm.ijpvm_76_23,38563037,
2,Differences in Biomechanical Determinants of A...,BACKGROUND: Change of direction (COD) movement...,Sports medicine - open,"Thomas A Donelon, Jamie Edwards, Mathew Brown,...",2024,,10,1,29,,10.1186/s40798-024-00701-z,38561438,
3,The Effectiveness and Optimal Dose of Resistan...,A subgroup of patients with low back pain (LBP...,Cureus,"Valerio Barbari, Maria M Carbone, Lorenzo Stor...",2024,,16,3,e57278,,10.7759/cureus.57278,38559546,
4,Sex differences in the physiological responses...,BACKGROUND: Heart disease is one of the leadin...,"BMC sports science, medicine &amp; rehabilitation","J Bouakkar, T J Pereira, H Johnston, M Pakosh,...",2024,,16,1,74,,10.1186/s13102-024-00867-9,38549168,


In [35]:
result_dict[iteration].PMIDs_dict

{1: ['38563729', '38563037'],
 2: ['38568258', '38565633'],
 3: ['38563729', '38563037', '38561438', '38559546', '38549168']}

In [36]:
result_dict[iteration].responses_dict

{1: {'header': {'type': 'esearch', 'version': '0.3'},
  'esearchresult': {'count': '2537',
   'retmax': '2',
   'retstart': '0',
   'idlist': ['38563729', '38563037'],
   'translationset': [{'from': 'y_10[Filter]', 'to': '"last 10 years"[dp]'},
    {'from': 'meta-analysis[Filter]', 'to': 'meta-analysis [PT]'},
    {'from': 'review[Filter]', 'to': 'review [PT]'}],
   'querytranslation': '"resistance train*"[All Fields] AND ("2014/04/03 00:00":"3000/01/01 05:00"[Date - Publication] AND ("meta analysis"[Publication Type] OR "review"[Publication Type] OR "systematic review"[Filter]))'}},
 2: {'header': {'type': 'esearch', 'version': '0.3'},
  'esearchresult': {'count': '19573',
   'retmax': '2',
   'retstart': '0',
   'idlist': ['38568258', '38565633'],
   'translationset': [],
   'querytranslation': '"resistance train*"[All Fields]'}},
 3: {'header': {'type': 'esearch', 'version': '0.3'},
  'esearchresult': {'count': '369',
   'retmax': '5',
   'retstart': '0',
   'idlist': ['38563729', '

Next: 
* Extract mesh headings
* Updating logging messages

# *End of Page*